In [1]:
import json
import pysal as ps
import urllib
import numpy as np

In [2]:
from urlparse import urlparse
def parse_geojson(resource):
    gj = urlparse(resource)
    if not gj[1]:
        # either a serialized object or a local file
        try:
            return json.loads(gj[2])
        except:
            with open(gj[2]) as f:
                return json.load(f)    
    else:
        # remote url
        fs = urllib.urlopen(resource)
        return json.load(fs)
    return gj

r = parse_geojson("http://toae.org/pub/columbus.json")
ss = json.dumps(r)
s = parse_geojson(ss)
lf = parse_geojson("../examples/columbus.json")



In [3]:
lf == s and s == r

True

In [4]:
def contiguity_from_geojson(gj_input, wtype='rook'):
    WTYPE = wtype.upper()
    if WTYPE not in ['QUEEN', 'ROOK']:
        print "wtype must be 'QUEEN' or 'ROOK'"
    WTYPE =["QUEEN", 'ROOK'].index(WTYPE)+1
    collection = parse_geojson(gj_input)   
    n = len(collection)
    collection_type = collection['features'][0]['geometry']['type']
    if collection_type in ['Polygon', 'MultiPolygon']:
        polys = []
        ids = []
        i = 0
        for feature in collection['features']:
            polys.append(ps.cg.asShape(feature['geometry']))
            ids.append(i)
            i +=1
        pcollection = ps.cg.shapes.PolygonCollection(dict(zip(ids,polys)))
        neighbors = ps.weights.Contiguity.ContiguityWeightsPolygons(pcollection, wttype=WTYPE).w
        return ps.W(neighbors)
    else:
        print "GeoJSON feature type must be 'Polygon' or 'MultiPolygon'"
       
def queen_from_geojson(uri):
    return contiguity_from_geojson(uri, wtype='QUEEN')

def rook_from_geojson(uri):
    return contiguity_from_geojson(uri, wtype="ROOK")
    


In [5]:
wql = queen_from_geojson("../examples/columbus.json")
wqr = queen_from_geojson("http://toae.org/pub/columbus.json")
ss = json.dumps(json.load(urllib.urlopen("http://toae.org/pub/columbus.json")),separators=(',',':') )
wqs = queen_from_geojson(ss)

setting bounding box for FeatureCollection
setting bounding box for FeatureCollection
setting bounding box for FeatureCollection


In [6]:
wql.neighbors == wqr.neighbors == wqs.neighbors

True

In [7]:
# point FeatureCollection
baltimore = "../examples/baltimore.geojson"
collection = parse_geojson(baltimore)



In [8]:
def get_coordinates_from_collection(collection):
    coordinates = []
    for feature in collection['features']:
        coordinates.append(feature['geometry']['coordinates'])
    return np.array(coordinates)



In [9]:
get_coordinates_from_collection(collection)

array([[ 907. ,  534. ],
       [ 922. ,  574. ],
       [ 920. ,  581. ],
       [ 923. ,  578. ],
       [ 918. ,  574. ],
       [ 900. ,  577. ],
       [ 918. ,  576. ],
       [ 907. ,  576. ],
       [ 918. ,  562. ],
       [ 897. ,  576. ],
       [ 916. ,  569. ],
       [ 908. ,  573. ],
       [ 913. ,  566. ],
       [ 910. ,  574. ],
       [ 922. ,  569. ],
       [ 913. ,  536. ],
       [ 919. ,  533.5],
       [ 917.5,  535. ],
       [ 933. ,  548.5],
       [ 932.5,  552.5],
       [ 936.5,  548.5],
       [ 930. ,  542.5],
       [ 925. ,  545. ],
       [ 927. ,  552. ],
       [ 936. ,  554.5],
       [ 860. ,  554. ],
       [ 868. ,  550.5],
       [ 872.5,  543. ],
       [ 880.5,  544.5],
       [ 869. ,  551.5],
       [ 883. ,  538. ],
       [ 876. ,  541. ],
       [ 875.5,  549. ],
       [ 875. ,  550. ],
       [ 868. ,  545. ],
       [ 879. ,  552. ],
       [ 860. ,  555.5],
       [ 868. ,  556.5],
       [ 873. ,  549. ],
       [ 888.5,  545. ],


In [10]:
def get_properites_from_collection(collection, properties=[]):
    if not properties:
        properties = collection['features'][0]['properties'].keys()
    values = []
    for feature in collection['features']:
        values.append([feature['properties'][prop] for prop in properties])
    return properties, values    
    

In [11]:
b = get_properites_from_collection(collection)

In [12]:
len(b)

2

In [13]:
b[0]

[u'NSTOR',
 u'AC',
 u'CITCOU',
 u'NROOM',
 u'SQFT',
 u'PRICE',
 u'GAR',
 u'NBATH',
 u'Y',
 u'STATION',
 u'FIREPL',
 u'BMENT',
 u'X',
 u'DWELL',
 u'LOTSZ',
 u'AGE',
 u'PATIO']

In [14]:
b[1]

[[3.0,
  0.0,
  0.0,
  4.0,
  11.25,
  47.0,
  0.0,
  1.0,
  534.0,
  1,
  0.0,
  2.0,
  907.0,
  0.0,
  5.7,
  148.0,
  0.0],
 [2.0,
  1.0,
  1.0,
  7.0,
  28.92,
  113.0,
  2.0,
  2.5,
  574.0,
  2,
  1.0,
  2.0,
  922.0,
  1.0,
  279.51,
  9.0,
  1.0],
 [2.0,
  0.0,
  1.0,
  7.0,
  30.62,
  165.0,
  2.0,
  2.5,
  581.0,
  3,
  1.0,
  3.0,
  920.0,
  1.0,
  70.64,
  23.0,
  1.0],
 [2.0,
  1.0,
  1.0,
  7.0,
  26.12,
  104.3,
  2.0,
  2.5,
  578.0,
  4,
  1.0,
  2.0,
  923.0,
  1.0,
  174.63,
  5.0,
  1.0],
 [2.0,
  0.0,
  1.0,
  7.0,
  22.04,
  62.5,
  0.0,
  1.5,
  574.0,
  5,
  1.0,
  2.0,
  918.0,
  1.0,
  107.8,
  19.0,
  1.0],
 [3.0,
  0.0,
  1.0,
  6.0,
  39.42,
  70.0,
  1.0,
  2.5,
  577.0,
  6,
  1.0,
  3.0,
  900.0,
  1.0,
  139.64,
  20.0,
  1.0],
 [1.0,
  1.0,
  1.0,
  6.0,
  21.88,
  127.5,
  2.0,
  2.5,
  576.0,
  7,
  1.0,
  3.0,
  918.0,
  1.0,
  250.0,
  20.0,
  1.0],
 [3.0,
  0.0,
  1.0,
  8.0,
  36.72,
  53.0,
  0.0,
  1.5,
  576.0,
  8,
  0.0,
  0.0,
  907.0,
  1.

In [15]:
b = get_properites_from_collection(collection, ['AGE', 'AC'])

In [16]:
b

(['AGE', 'AC'],
 [[148.0, 0.0],
  [9.0, 1.0],
  [23.0, 0.0],
  [5.0, 1.0],
  [19.0, 0.0],
  [20.0, 0.0],
  [20.0, 1.0],
  [22.0, 0.0],
  [22.0, 1.0],
  [4.0, 1.0],
  [23.0, 0.0],
  [20.0, 1.0],
  [30.0, 0.0],
  [20.0, 1.0],
  [18.0, 0.0],
  [75.0, 0.0],
  [60.0, 0.0],
  [65.0, 0.0],
  [14.0, 0.0],
  [45.0, 0.0],
  [14.0, 0.0],
  [22.0, 0.0],
  [35.0, 0.0],
  [5.0, 1.0],
  [60.0, 1.0],
  [14.0, 1.0],
  [19.0, 1.0],
  [11.0, 0.0],
  [16.0, 1.0],
  [19.0, 1.0],
  [17.0, 0.0],
  [24.0, 0.0],
  [22.0, 0.0],
  [27.0, 0.0],
  [3.0, 1.0],
  [21.0, 1.0],
  [20.0, 0.0],
  [4.0, 1.0],
  [24.0, 0.0],
  [40.0, 0.0],
  [23.0, 0.0],
  [35.0, 0.0],
  [25.0, 0.0],
  [17.0, 1.0],
  [26.0, 1.0],
  [15.0, 1.0],
  [29.0, 0.0],
  [24.0, 1.0],
  [30.0, 0.0],
  [19.0, 0.0],
  [50.0, 0.0],
  [18.0, 1.0],
  [74.0, 0.0],
  [22.0, 1.0],
  [80.0, 0.0],
  [24.0, 1.0],
  [20.0, 1.0],
  [7.0, 1.0],
  [47.0, 0.0],
  [50.0, 0.0],
  [45.0, 0.0],
  [48.0, 1.0],
  [50.0, 0.0],
  [50.0, 0.0],
  [48.0, 0.0],
  [55.0, 0.0],


In [17]:
def knn_from_geojson(gj, k=2):
    collection = parse_geojson(gj)
    return ps.knnW_from_array(get_coordinates_from_collection(collection),k)
    
    


In [18]:
knn_balt = knn_from_geojson(baltimore, k=5)

In [19]:
knn_balt.n

211

In [20]:
knn_balt.neighbors

{0: [95, 15, 89, 132, 177],
 1: [4, 3, 6, 184, 14],
 2: [3, 6, 4, 1, 184],
 3: [1, 2, 6, 4, 184],
 4: [6, 1, 10, 14, 3],
 5: [9, 69, 7, 11, 13],
 6: [4, 1, 3, 2, 10],
 7: [11, 13, 69, 5, 9],
 8: [194, 75, 210, 67, 12],
 9: [5, 69, 47, 7, 11],
 10: [12, 4, 14, 8, 6],
 11: [13, 7, 69, 66, 12],
 12: [10, 66, 64, 68, 8],
 13: [11, 7, 10, 4, 69],
 14: [179, 1, 184, 194, 10],
 15: [17, 178, 95, 0, 16],
 16: [17, 178, 73, 134, 138],
 17: [16, 73, 178, 15, 72],
 18: [20, 19, 24, 21, 23],
 19: [24, 18, 199, 23, 20],
 20: [18, 202, 19, 24, 199],
 21: [203, 71, 204, 22, 18],
 22: [71, 74, 21, 23, 72],
 23: [74, 19, 18, 191, 190],
 24: [199, 19, 20, 18, 186],
 25: [36, 37, 26, 45, 29],
 26: [29, 38, 34, 37, 33],
 27: [31, 34, 38, 32, 33],
 28: [41, 31, 32, 30, 35],
 29: [26, 38, 37, 33, 34],
 30: [85, 165, 28, 169, 83],
 31: [27, 28, 152, 30, 32],
 32: [33, 38, 35, 27, 28],
 33: [32, 38, 35, 29, 26],
 34: [27, 26, 38, 29, 32],
 35: [33, 32, 46, 76, 38],
 36: [25, 37, 45, 26, 29],
 37: [45, 49, 29,

In [21]:
def centroid(coordinates):
    x = [coord[0] for coord in coordinates]
    y = [coord[1] for coord in coordinates]
    n = len(x)
    cx = 0
    cy = 0
    A = 0
    for i in xrange(n-1):
        A += (x[i] * y[i+1] - x[i+1] * y[i])
        f = (x[i] * y[i + 1] - x[i + 1] * y[i])
        cx += (x[i] + x[i + 1]) * f
        cy += (y[i] + y[i + 1]) * f
    A /= 2.0
    A6 = 1./(6. * A)
    cx *= A6
    cy *= A6
    return (cx, cy)

In [22]:
columbus = ps.examples.get_path("columbus.json")
pc = parse_geojson(columbus)
for feature in pc['features']:
    coordinates = feature['geometry']['coordinates']
    ring = coordinates[0]
    print centroid(ring)

(8.827218469203899, 14.36907601891205)
(8.332658372724943, 14.031624006323977)
(9.012265409371706, 13.819719079969202)
(8.460800893307436, 13.716962170228118)
(9.007982127497668, 13.296366430537311)
(9.739925828001203, 13.474632743210663)
(8.11875006062871, 13.295703867053506)
(8.496488171957564, 13.402612361972402)
(9.630793442508978, 12.942724931765394)
(10.366382709801035, 13.001889360253092)
(8.669735412857024, 12.980121561233249)
(8.544995589641758, 12.953125683982284)
(8.349223482851091, 12.99609414110248)
(8.292701516920962, 12.863707688216877)
(8.973462206070076, 12.741586740380432)
(8.65586633556042, 12.627315808754112)
(10.528621192777125, 12.64747559705899)
(8.487918125725216, 12.545342471590114)
(8.312367703365659, 12.666720813263007)
(10.190581526179404, 12.256899149288603)
(7.847423618980442, 12.084997322022756)
(9.65524436749835, 12.462765864102991)
(10.671381108158535, 12.291571317666149)
(8.4207440682731, 12.318013434287632)
(8.938751825549424, 12.380509826386794)
(9.2

In [23]:
def get_points_array_from_geojson(gj):
    collection = parse_geojson(gj)
    # check type of first feature
    stype = collection['features'][0]['geometry']['type']
    if stype == 'Point':
        return np.array(get_coordinates_from_collection(collection))
    else: #polygon or multipolygon
        centroids = []
        for feature in collection['features']:
            coordinates = feature['geometry']['coordinates']
            centroids.append(centroid(coordinates[0]))   # use external ring for poly, or first ring for mp 
        k = len(centroids[0])
        return np.array(centroids)
    

In [24]:
p = get_points_array_from_geojson(baltimore)

In [25]:

pc = get_points_array_from_geojson(columbus)


In [26]:
pc.shape

(49, 2)

In [27]:
def knn_from_geojson(gj, k=2):
    return ps.knnW_from_array(get_points_array_from_geojson(gj),k)
    
    


In [28]:
knn_from_geojson(columbus)

In [29]:
knn_from_geojson(baltimore)

In [30]:
knn2 = knn_from_geojson(columbus)

In [31]:
knn2.n

49

In [32]:
knn2.weights

{0: [1.0, 1.0],
 1: [1.0, 1.0],
 2: [1.0, 1.0],
 3: [1.0, 1.0],
 4: [1.0, 1.0],
 5: [1.0, 1.0],
 6: [1.0, 1.0],
 7: [1.0, 1.0],
 8: [1.0, 1.0],
 9: [1.0, 1.0],
 10: [1.0, 1.0],
 11: [1.0, 1.0],
 12: [1.0, 1.0],
 13: [1.0, 1.0],
 14: [1.0, 1.0],
 15: [1.0, 1.0],
 16: [1.0, 1.0],
 17: [1.0, 1.0],
 18: [1.0, 1.0],
 19: [1.0, 1.0],
 20: [1.0, 1.0],
 21: [1.0, 1.0],
 22: [1.0, 1.0],
 23: [1.0, 1.0],
 24: [1.0, 1.0],
 25: [1.0, 1.0],
 26: [1.0, 1.0],
 27: [1.0, 1.0],
 28: [1.0, 1.0],
 29: [1.0, 1.0],
 30: [1.0, 1.0],
 31: [1.0, 1.0],
 32: [1.0, 1.0],
 33: [1.0, 1.0],
 34: [1.0, 1.0],
 35: [1.0, 1.0],
 36: [1.0, 1.0],
 37: [1.0, 1.0],
 38: [1.0, 1.0],
 39: [1.0, 1.0],
 40: [1.0, 1.0],
 41: [1.0, 1.0],
 42: [1.0, 1.0],
 43: [1.0, 1.0],
 44: [1.0, 1.0],
 45: [1.0, 1.0],
 46: [1.0, 1.0],
 47: [1.0, 1.0],
 48: [1.0, 1.0]}

In [33]:
fc = parse_geojson(columbus)
def add_W_2_feature_collection(fc, w, wname):
    for i,feature in enumerate(fc['features']):
        properties = feature['properties']
        if 'weights' not in properties.keys():
            weights = {}
            properties[u'weights'] = weights
        weights = properties['weights']
        try:
            weights[wname] = [list(w.neighbors[i]),w.weights[i]]
        except:
            weights[wname] = [w.neighbors[i],w.weights[i]]
        properties['weights'] = weights
        feature['properties'] = properties
        fc['features'][i] = feature
    return fc

In [34]:
fcadd = add_W_2_feature_collection(fc, knn2, "knn2")

In [35]:
fcadd

{u'features': [{u'bbox': [8.559700012207031,
    13.995059967041016,
    9.09996509552002,
    14.742449760437012],
   u'geometry': {u'coordinates': [[[8.624129295349121, 14.236980438232422],
      [8.559700012207031, 14.742449760437012],
      [8.809452056884766, 14.734430313110352],
      [8.808412551879883, 14.636520385742188],
      [8.919304847717285, 14.638500213623047],
      [9.087138175964355, 14.63049030303955],
      [9.09996509552002, 14.244830131530762],
      [9.015047073364258, 14.241840362548828],
      [9.008951187133789, 13.995059967041016],
      [8.818140029907227, 14.002050399780273],
      [8.653305053710938, 14.008090019226074],
      [8.642902374267578, 14.089710235595703],
      [8.63259220123291, 14.1705904006958],
      [8.625825881958008, 14.22367000579834],
      [8.624129295349121, 14.236980438232422]]],
    u'type': u'Polygon'},
   u'properties': {u'AREA': 0.309441,
    u'COLUMBUS_': 2.0,
    u'COLUMBUS_I': 5.0,
    u'CP': 0.0,
    u'CRIME': 15.72598,
   

In [36]:
wqc = queen_from_geojson(columbus)



setting bounding box for FeatureCollection


In [37]:
fcadd = add_W_2_feature_collection(fcadd, wqc, 'wqc')

In [38]:
fcadd

{u'features': [{u'bbox': [8.559700012207031,
    13.995059967041016,
    9.09996509552002,
    14.742449760437012],
   u'geometry': {u'coordinates': [[[8.624129295349121, 14.236980438232422],
      [8.559700012207031, 14.742449760437012],
      [8.809452056884766, 14.734430313110352],
      [8.808412551879883, 14.636520385742188],
      [8.919304847717285, 14.638500213623047],
      [9.087138175964355, 14.63049030303955],
      [9.09996509552002, 14.244830131530762],
      [9.015047073364258, 14.241840362548828],
      [9.008951187133789, 13.995059967041016],
      [8.818140029907227, 14.002050399780273],
      [8.653305053710938, 14.008090019226074],
      [8.642902374267578, 14.089710235595703],
      [8.63259220123291, 14.1705904006958],
      [8.625825881958008, 14.22367000579834],
      [8.624129295349121, 14.236980438232422]]],
    u'type': u'Polygon'},
   u'properties': {u'AREA': 0.309441,
    u'COLUMBUS_': 2.0,
    u'COLUMBUS_I': 5.0,
    u'CP': 0.0,
    u'CRIME': 15.72598,
   

In [39]:
fcadds = json.dumps(fcadd)

In [40]:
fcread = parse_geojson(fcadds)

In [43]:
def get_weights_feature_collection(gj):
    f0 = gj['features'][0]
    if 'weights' in f0['properties']:
        n_w = len(f0['properties']['weights'])
        w_names = f0['properties']['weights'].keys()
        info = dict([(w_name,{'weights':{},"neighbors":{}}) for w_name in w_names])
        #print info
        for i, feature in enumerate(gj['features']):
            for name in w_names:
                #print feature['properties']['weights'][name][0]
                neighbors = feature['properties']['weights'][name][0]
                weights = feature['properties']['weights'][name][1]
                info[name]['neighbors'][i] = neighbors
                info[name]['weights'][i] = weights
        ws = {}
        for name in info.keys():
            ws[name] = ps.W(neighbors = info[name]['neighbors'],
                           weights = info[name]['weights'])
        return ws
            
            

In [44]:
weights = get_weights_feature_collection(fcread)

In [45]:
weights['knn2'].neighbors == knn2.neighbors  

True

In [48]:
weights['wqc'].neighbors[0] == list(wqc.neighbors[0]) #our continguity builder sets the neighbors values to a set not a list

True

In [49]:
wqc.neighbors

{0: {1, 2},
 1: {0, 2, 3},
 2: {0, 1, 3, 4},
 3: {1, 2, 4, 7},
 4: {2, 3, 5, 7, 8, 10, 14, 15},
 5: {4, 8},
 6: {7, 11, 12, 13},
 7: {3, 4, 6, 10, 11, 12},
 8: {4, 5, 9, 14, 19, 21, 24, 25},
 9: {8, 16, 19, 21},
 10: {4, 7, 11, 14, 15},
 11: {6, 7, 10, 12, 13, 15},
 12: {6, 7, 11, 13},
 13: {6, 11, 12, 15, 17, 18},
 14: {4, 8, 10, 15, 24, 25},
 15: {4, 10, 11, 13, 14, 17, 23, 24},
 16: {9, 19, 22},
 17: {13, 15, 18, 23},
 18: {13, 17, 23},
 19: {8, 9, 16, 21, 22, 26, 31, 32, 34, 39},
 20: {23, 29, 33},
 21: {8, 9, 19, 25, 26, 27},
 22: {16, 19, 31},
 23: {15, 17, 18, 20, 24, 28, 29},
 24: {8, 14, 15, 23, 25, 27, 28, 29},
 25: {8, 14, 21, 24, 27, 28},
 26: {19, 21, 27, 32},
 27: {21, 24, 25, 26, 28, 32, 34, 36, 37},
 28: {23, 24, 25, 27, 29, 36, 37},
 29: {20, 23, 24, 28, 36},
 30: {33, 35, 38},
 31: {19, 22, 39, 40},
 32: {19, 26, 27, 34},
 33: {20, 30, 35, 41},
 34: {19, 27, 32, 37, 39, 42, 43},
 35: {30, 33, 38, 41, 45},
 36: {27, 28, 29, 37, 42, 44},
 37: {27, 28, 34, 36, 42, 43},
 